In [3]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style("darkgrid")
#sns.set(rc={'figure.figsize':(8,6)}) # adjust figure size

#%precision 2

pd.set_option('max_columns',200)

# modeling packages
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [4]:
import datetime

# NSC data

In [30]:
# read the file
nsc = pd.read_excel('UtmostU NSC Data_For DePaul.xlsx', 'Sheet1')

# Data clean log

In [31]:
# make a copy
nsc_copy = nsc.copy()

In [32]:
nsc_copy.shape

(7234, 16)

### 1) Remove rows whose class level is *High School Scholar*

In [34]:
# drop rows whose class level is High School Scholar
nsc_copy = nsc_copy.drop(nsc_copy[(nsc_copy['Class Level'] == 'High School Scholar')].index)

In [35]:
nsc_copy.shape
# 17 rows removed

(7217, 16)

### 2) For *students* who have both advanced certificate and basic certificate, only keep advanced certificate 

In [36]:
certificate = nsc[nsc['Degree Title'].str.contains("CERT", na=False)]

In [37]:
# there are 40 rows in total. All these fellows got at least one certificate. 
# DIPLOMA --> certificate (00346000002iXcS) only one fellow
certificate.shape

(40, 16)

In [38]:
certificate['Degree Title'].value_counts()

BASIC CERTIFICATE            30
CERTIFICATE                   4
ADVANCED CERTIFICATE          3
UNDERGRADUATE CERTIFICATE     1
CERTIFICATE IN WRITING        1
CERT                          1
Name: Degree Title, dtype: int64

In [39]:
# find the fellows who earned advanced certificate
fellow_ad_cert = nsc.loc[nsc['Degree Title'] == 'ADVANCED CERTIFICATE']
fellow_ad_cert

,Fellow ID,College Code/Branch,College Name,College State,2-year / 4-year,Public / Private,Enrollment Begin,Enrollment End,Enrollment Status,Class Level,Enrollment Major 1,Graduated?,Graduation Date,Degree Title,Degree Major 1,Degree Major 2
376,0034p00001iGQ6m,001650-00,CITY OF CHICAGO - MALCOLM X COLLEGE,IL,2,Public,NaN,NaN,,NaN,NaN,Y,5/15/2021,ADVANCED CERTIFICATE,Education,NaN
982,0034600001EXcUB,001654-00,CITY OF CHICAGO - KENNEDY-KING COLLEGE,IL,2,Public,NaN,NaN,,NaN,NaN,Y,12/14/2019,ADVANCED CERTIFICATE,"Repair, Production, & Construction",NaN
2764,00346000002iXd2,009767-00,CITY OF CHICAGO - OLIVE-HARVEY COLLEGE,IL,2,Public,NaN,NaN,,NaN,NaN,Y,5/11/2019,ADVANCED CERTIFICATE,Education,NaN


In [40]:
# drop rows
nsc_copy = nsc_copy.drop(nsc_copy[(nsc_copy['Fellow ID'] == '0034p00001iGQ6m') & (nsc_copy['Degree Title'] == 'BASIC CERTIFICATE')].index)
nsc_copy = nsc_copy.drop(nsc_copy[(nsc_copy['Fellow ID'] == '0034600001EXcUB') & (nsc_copy['Degree Title'] == 'BASIC CERTIFICATE')].index)
nsc_copy = nsc_copy.drop(nsc_copy[(nsc_copy['Fellow ID'] == '00346000002iXd2') & (nsc_copy['Degree Title'] == 'BASIC CERTIFICATE')].index)

In [41]:
nsc_copy.shape
# 5 rows removed

(7212, 16)

### 3) remove rows which have graduation date but no degree type

In [42]:
graduated_fellow = nsc.loc[nsc['Graduated?'] == 'Y']

In [43]:
# graduated_fellow.head()

In [44]:
# fellows who have more than one graduation records
counts = graduated_fellow['Fellow ID'].value_counts()
graduated_fellow_more_than_one = graduated_fellow[graduated_fellow['Fellow ID'].isin(counts.index[counts > 1])]

In [45]:
graduated_fellow_more_than_one['Fellow ID'].unique()

array(['0034p00001huqzw', '00346000002iXbP', '00346000002iXf4',
       '0034p00001iGQ6m', '00346000002iXan', '00346000002iXes',
       '00346000002iXXW', '00346000002iXcp', '00346000002iXZM',
       '0034p00001iGQ5v', '00346000002iXZR', '0034600001EXcUB',
       '00346000002iXXU', '0034p00001iGQ61', '00346000002iXXI',
       '00346000002iXZF', '00346000002iXev', '00346000002iXZt',
       '00346000002iXco', '00346000002iXXB', '00346000002iXYx',
       '00346000002iXa3', '00346000002iWAj', '0034p00001jVylT',
       '00346000002iWOw', '00346000002iXd2', '00346000002iXZZ',
       '00346000002iW9n', '00346000002iXaZ', '00346000002iXbo',
       '00346000002iXcJ', '00346000002iXaO', '0034p00001huJkF',
       '00346000002iXZb', '00346000002iXbZ', '00346000002iXaD',
       '00346000002iXbN', '00346000002iXZc', '00346000002iXZq',
       '00346000002iWAF', '00346000002iXdX', '0034p00001iGQ5e',
       '00346000002iXaE', '00346000002iXcP', '0034600000iXknH',
       '00346000002iXbL', '0034600001EXc

**We found that if there is a graduation date but no degree title, in most cases, this is because the graduation date is trying to record the last date of the last semester. There will be another row recording both actuall graduation date and degree title. Therefore, we can remove those rows which have graduation date but no degree title.**

In [46]:
# drop those rows which have a graduation date but no degree title
nsc_copy = nsc_copy.drop(nsc_copy[(nsc_copy['Graduated?'] == 'Y') & (nsc_copy['Degree Title'].isnull())].index)

In [47]:
nsc_copy.shape
# 39 rows removed

(7173, 16)

**the following fellows are removed:**
['0034p00001jVEU7', '0034600001EXcSa', '00346000002iWAv', '00346000002iXbv', '00346000002iXeF']

*they only have one record which has a graduation date but no degree type. They cannot be used to compute duration anyways. So removed.*

**We also found fellow (00346000002iXcJ) has two graduation dates with the same degree title. One of teh rows should be removed**

In [48]:
# 00346000002iXcJ
# remove index 3371
nsc_copy.drop(3371, inplace=True)

### 4) remove very close graduation dates

**I used date diff to find close graduation dates, and found those are**
- Reduplications
- a degree with a certificate

In [ ]:
# teh following fellows have this situation
#['0034p00001huqzw', '0034p00001iGQ61', '00346000002iXco',
#'00346000002iXaZ', '0034p00001huJkF', '00346000002iXdX',
#'0034600001EXcSe', '00346000002iWAc', '00346000002iXf6',
#'00346000002iXXo', '00346000002iWAA']

In [49]:
# See example below
temp = nsc_copy.loc[nsc_copy['Fellow ID'] == '0034p00001huqzw']
temp

,Fellow ID,College Code/Branch,College Name,College State,2-year / 4-year,Public / Private,Enrollment Begin,Enrollment End,Enrollment Status,Class Level,Enrollment Major 1,Graduated?,Graduation Date,Degree Title,Degree Major 1,Degree Major 2
44,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,8/26/2019,12/14/2019,Full-time,Freshman,NaN,N,NaN,NaN,NaN,NaN
45,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,1/13/2020,5/9/2020,Full-time,Sophomore,NaN,N,NaN,NaN,NaN,NaN
46,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,8/24/2020,12/12/2020,Full-time,Sophomore,NaN,N,NaN,NaN,NaN,NaN
47,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,1/19/2021,5/15/2021,Full-time,Sophomore,NaN,N,NaN,NaN,NaN,NaN
48,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,NaN,NaN,,NaN,NaN,Y,5/15/2021,ASSOCIATE IN ARTS,"Accounting, Economics & Finance",NaN
49,0034p00001huqzw,001649-00,CITY OF CHICAGO - RICHARD J. DALEY COLLEGE,IL,2,Public,NaN,NaN,,NaN,NaN,Y,5/15/2021,GECC CREDENTIAL,General Studies,NaN


In [50]:
# the following rows should be removed:
#  [49,1494, 2119, 3070, 3493, 4533, 4964, 5684, 6520, 6521, 6799, 7099, 7100, 7101]
remove_index = [49,1494, 2119, 3070, 3493, 4533, 4964, 5684, 6520, 6521, 6799, 7099, 7100, 7101]

In [51]:
nsc_copy.drop(remove_index, inplace=True)

In [52]:
nsc_copy.shape
# 14 rows removed

(7158, 16)

**conclusion:**
- removed 17 rows --> high school scholar
- removed 5 rows --> basic certificate
- removed 39 rows --> have graduation date but no degree type (most are reduplications)
- removed 1 row --> have two close graduation dates, but same degree title
- removed 14 rows --> same graduation date (with a less important degree such as certificate, credential, minor degree etc.)

In [54]:
# write as csv file
# nsc_copy.to_csv('NSC_new_version_1026.csv', encoding='utf-8', index=False)